
### Target:
1. Acheive accuracy of 99.4 by involving image augmentation and if required by reduction of the learning rate 
2. Make sure the parameters are below 8k and 15 epochs
3. The model changes will be minimal and more will be on images and the learning optimizers.

### Results:
Parameters:7,216  
Best Train Accuracy: 99.19 % (15th Epoch)   
Best Test Accuracy: 99.45% (15th Epoch)   

### Analysis:
1. We are using the model 4 from the previous step to build model of accuracy above 99.4%
2. On the code 5, we are applying image augmentation by rotating the mnist image by angle of rotation between -7.0 and 7.0 degrees. The image augmentation transformation is only applied on the training dataset and not on test dataset
3. The model overfitting issues were completely sorted and we can see the gap between the training and testing accuracy was reducing, but the accuracy was not still hitting the 99.4 mark
4. To improve the model accuracy again, we introduced reducelronplateau as the learning rate scheduler method
5. ReduceLRonPlateau was set as patience as 2. This means that the LR will be same for the first 2 epochs with no improvement, and will only decrease the LR after the 3rd epoch if the loss still hasn’t improved 
6. For ReduceLRonPlateau, we are calling separate function for the testing, because for the learning rate change we have to pass the test loss. The function is specified in the utils.py as test_model_plateau
7. It is visible that the training set slowly hits the 99% accuracy in the last few epochs whilst the testing accuracy crosses the 99.4 mark because of the learning rate change. 

In [1]:
#!pip install torchsummary
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from utils import data_loader,data_statistics,modelsummary,train_model,test_model,test_model_plateau
from model import *

In [2]:
# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)
device = torch.device("cuda" if cuda else "cpu")

CUDA Available? True


In [3]:
# Train Phase transformations
train_transforms=transforms.Compose([
                        transforms.RandomRotation((-7.0, 7.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])
# Test Phase transformations
test_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])


In [4]:
batch_size=128
train_loader,train,test_loader,test= data_loader(train_transforms,test_transforms,batch_size)

100%|██████████| 9912422/9912422 [00:00<00:00, 425387894.37it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 21026851.90it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 77272529.57it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 5245189.64it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

CUDA Available? True


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


#### Code 5- Model 4 - Apply image augmentatioon

In [ ]:
model =  Model_4().to(device)
modelsummary(model)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
              ReLU-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
           Dropout-4            [-1, 8, 28, 28]               0
            Conv2d-5           [-1, 12, 28, 28]             876
              ReLU-6           [-1, 12, 28, 28]               0
       BatchNorm2d-7           [-1, 12, 28, 28]              24
           Dropout-8           [-1, 12, 28, 28]               0
            Conv2d-9            [-1, 6, 28, 28]              78
        MaxPool2d-10            [-1, 6, 14, 14]               0
           Conv2d-11           [-1, 12, 12, 12]             660
             ReLU-12           [-1, 12, 12, 12]               0
      BatchNorm2d-13           [-1, 12, 12, 12]              24
          Dropout-14           [-1, 12,

/content/model.py:155: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch+1)
    train_model(model, device, train_loader, optimizer, epoch)
    test_model(model, device, test_loader)

EPOCH: 1


Loss=0.20094235241413116 Batch_id=468 Accuracy=89.00: 100%|██████████| 469/469 [00:43<00:00, 10.79it/s]



Test set: Average loss: 0.0781, Accuracy: 9772/10000 (97.72%)

EPOCH: 2


Loss=0.06386285275220871 Batch_id=468 Accuracy=96.90: 100%|██████████| 469/469 [00:44<00:00, 10.58it/s]



Test set: Average loss: 0.0434, Accuracy: 9867/10000 (98.67%)

EPOCH: 3


...


Test set: Average loss: 0.0264, Accuracy: 9917/10000 (99.17%)

EPOCH: 14


Loss=0.08474791795015335 Batch_id=468 Accuracy=98.68: 100%|██████████| 469/469 [00:44<00:00, 10.54it/s]



Test set: Average loss: 0.0261, Accuracy: 9913/10000 (99.13%)

EPOCH: 15


Loss=0.037887413054704666 Batch_id=468 Accuracy=98.66: 100%|██████████| 469/469 [00:46<00:00, 10.06it/s]



Test set: Average loss: 0.0227, Accuracy: 9930/10000 (99.30%)



#### Code 6- Model 4-Reduce LR on Plateau

In [5]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
model =  Model_4().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = ReduceLROnPlateau(optimizer, patience=2)

EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch+1)
    train_model(model, device, train_loader, optimizer, epoch)
    test_loss = test_model_plateau(model, device, test_loader)
    scheduler.step(test_loss)

EPOCH: 1


  0%|          | 0/469 [00:00<?, ?it/s]/content/model.py:155: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
Loss=0.15595975518226624 Batch_id=468 Accuracy=90.43: 100%|██████████| 469/469 [00:29<00:00, 16.15it/s]



Test set: Average loss: 0.0795, Accuracy: 9773/10000 (97.73%)

EPOCH: 2


Loss=0.040991537272930145 Batch_id=468 Accuracy=97.40: 100%|██████████| 469/469 [00:20<00:00, 22.61it/s]



Test set: Average loss: 0.0435, Accuracy: 9871/10000 (98.71%)

EPOCH: 3


...


Test set: Average loss: 0.0193, Accuracy: 9942/10000 (99.42%)

EPOCH: 14


Loss=0.007884729653596878 Batch_id=468 Accuracy=99.13: 100%|██████████| 469/469 [00:20<00:00, 22.46it/s]



Test set: Average loss: 0.0185, Accuracy: 9944/10000 (99.44%)

EPOCH: 15


Loss=0.04473526403307915 Batch_id=468 Accuracy=99.19: 100%|██████████| 469/469 [00:20<00:00, 22.43it/s]



Test set: Average loss: 0.0186, Accuracy: 9945/10000 (99.45%)

